In [2]:
using DelimitedFiles

A = readdlm("sketches/ww15mgh.grd", skipstart = 1)
A1 = permutedims(A); #transpose into column major order
a1 = vec(A1); #concatenate into a vector to remove unwanted string elements
a1_clean = filter(x -> !(x isa AbstractString), a1);
a_f32 = convert(Array{Float32, 1}, a1_clean) #cast to Float32
#we know that each sequence of 1441 elements corresponds to a constant-latitude
#set of values. after reshaping, each column is a constant-latitude set of
#values running over lon [-pi, pi]
A_F32 = reshape(a_f32, 1441, :); 
A_F32 = permutedims(A_F32); #transpose so that each row corresponds to a latitude value
#in the original text file latitude decreases from π/2 to -π/2, we need to flip
#it!
A_final = reverse(A_F32, dims = 1);

#write as big endian
open("ww15mgh_be.bin", "w") do f #opens the specified file, returns it as f, and applies to it the function in the block 
    write(f, hton.(A_final))
end

#write as little endian
open("ww15mgh_le.bin", "w") do f #opens the specified file, returns it as f, and applies to it the function in the block 
    write(f, htol.(A_final))
end

#read from big endian
A_tmp = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh_be.bin", "r") do f
    read!(f, A_tmp)
end
A_retrieved = ntoh.(A_tmp)
@assert all(A_retrieved[end,:] .== Float32(13.606))
@assert all(A_retrieved[1,:] .== Float32(-29.534))

A_tmp = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh_le.bin", "r") do f
    read!(f, A_tmp)
end
A_retrieved = ltoh.(A_tmp)
@assert all(A_retrieved[end,:] .== Float32(13.606))
@assert all(A_retrieved[1,:] .== Float32(-29.534))

ArgumentError: ArgumentError: Cannot open 'sketches/ww15mgh.grd': not a file

In [12]:
B = []